<a href="https://colab.research.google.com/github/annamalaikannan/INFOSYS-INTERNSHIP/blob/main/Amazon_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [ ]:
import schedule
import time
from datetime import datetime, timedelta

In [ ]:
def get_title(soup):
    title=soup.find('span', {'id':'productTitle'})
    if title:
        return title.text.strip()
    else:
        return None

In [ ]:
def get_before_price(soup):
    price_before=soup.find('span', {'class':'a-price a-text-price'})
    if price_before:
        text = price_before.text.strip()
        result = text.split("₹")[1]
        result = "₹" + result
        return result
    else:
        return None

In [ ]:
def get_after_price(soup):
    price_after=soup.find('span', {'class':'a-price-whole'})
    if price_after:
        text = price_after.text.strip('.')
        result = "₹" + text
        return result
    else:
        return None

In [ ]:
def get_disc(soup):
    disc=soup.find('span', {'class':'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'})
    if disc:
        text = disc.text.lstrip('-')
        return text.split()[0]
    else:
        return None

In [ ]:
def get_rating(soup):
    rating=soup.find('span', {'class':'a-icon-alt'})
    if rating:
        text = rating.text.strip()
        return text.split()[0]
    else:
        return None

In [ ]:
def get_reviews(soup):
    reviews=soup.find('span', {'id':'acrCustomerReviewText'})
    if reviews:
        text = reviews.text.strip()
        return text.split()[0]
    return None

In [ ]:
def get_avail(soup):
    avail=soup.find('div', {'id':'availability'})
    return avail.text.strip() if avail else 'Not Available'

In [ ]:
def web_scraping():
    base_url = 'https://www.amazon.in'
    url='https://www.amazon.in/s?k=headphones&rh=p_n_availability%3A1318485031&dc&ds=v1%3AoWtjLHStC549MFFBdKBGJHo3n2XoMJZElN3kgJauGjQ&qid=1734452649&rnid=1318483031&ref=sr_nr_p_n_availability_2'
    headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    cookies=response.cookies
    soup=BeautifulSoup(response.content, 'html.parser')
    links=soup.find_all('a', attrs={'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal' })
    links_list=[link.get('href') for link in links]

    data={'Title':[], 'Price before':[], 'Price after':[], 'Discount':[], 'Rating':[], 'Reviews':[], 'Available/not':[]}

    for link in range(min(15, len(links_list))):
        new_page=requests.get(base_url+links_list[link], headers=headers, cookies=cookies)
        new_soup=BeautifulSoup(new_page.content, 'html.parser')

        data['Title'].append(get_title(new_soup))
        data['Price before'].append(get_before_price(new_soup))
        data['Price after'].append(get_after_price(new_soup))
        data['Discount'].append(get_disc(new_soup))
        data['Rating'].append(get_rating(new_soup))
        data['Reviews'].append(get_reviews(new_soup))
        data['Available/not'].append(get_avail(new_soup))

    df=pd.DataFrame(data)
    df.to_csv('amazon_final.csv', index=False)

    time_stamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"amazon_final_{time_stamp}.csv"

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")


In [ ]:
def scrape_amazon():
    print('Scraping Amazon...')
    web_scraping()

schedule.every(2).hours.do(scrape_amazon)

while True:
    schedule.run_pending()
    time.sleep(60)